# Embeddings

1. Использовать предобученную модель word2vec с сайта RusVectōrēs для решения задач: поиска n ближайших по смыслу слов, сложения и вычитания смысловых векторов, нахождения лишнего слова.
2. Провести обучение своей модели с использованием датасета с размеченными и неразмеченными отзывами о фильмах.
3. Провести дообучение существующей модели.
4. Выполнить оценку качества модели.
5. Выполнить t-SNE преобразование (без SVD и с SVD) визуализацию модели.
6. Использовать технологию fasttext, обучить модель и аналогично выполнить задачи: получения ближайших соседей, аналогий.
7. Решить задачу анализа тональности твитов с применением описанных выше технологий.

In [3]:
import urllib.request
import gensim
from gensim.models import word2vec
import zipfile

In [4]:
urllib.request.urlretrieve(
"https://vectors.nlpl.eu/repository/20/220.zip",
"ruwikiruscorpora_upos_cbow_300_10_2021.zip"
)

with zipfile.ZipFile(
 "ruwikiruscorpora_upos_cbow_300_10_2021.zip", 'r'
 ) as zip_ref:
 zip_ref.extractall("model")

KeyboardInterrupt: 

In [5]:
model_path = 'model/model.bin'

model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

Посмотрим на ближайших соседей следующей группы слов:

In [6]:
words = ['день_NOUN', 'ночь_NOUN', 'человек_NOUN', 'семантика_NOUN', 'биткоин_NOUN']

In [7]:
for word in words:
    if word in model_ru:
        print(word)
        print(model_ru[word][:10])
        for word, sim in model_ru.most_similar(positive=[word], topn=10):
            print(word, ': ', sim)
        print('\n')
    else:
        print('Увы, слова "%s" нет в модели!' % word)

день_NOUN
[ 1.607006    0.88105875 -0.90222335  0.1547316   2.5412288  -0.7520914
  1.7317989   0.5731876  -2.1542902  -2.353275  ]
неделя_NOUN :  0.767143726348877
месяц_NOUN :  0.7355299592018127
утро_NOUN :  0.6559638381004333
час_NOUN :  0.6324191689491272
ночь_NOUN :  0.5809687972068787
сутки_NOUN :  0.5776057243347168
вечер_NOUN :  0.5675091743469238
минута_NOUN :  0.548324465751648
день»_PROPN :  0.5270609259605408
денька_NOUN :  0.515771210193634


ночь_NOUN
[-1.9102603  -1.5677836   0.5107194   0.7136092  -1.7047127  -4.622845
  3.485842    1.2509006   0.04754175 -1.8432467 ]
вечер_NOUN :  0.7615349292755127
утро_NOUN :  0.7531794309616089
рассвет_NOUN :  0.727114737033844
полночь_NOUN :  0.6713059544563293
полдень_NOUN :  0.6476588249206543
сумерки_NOUN :  0.60179603099823
утр_NOUN :  0.5851607322692871
день_NOUN :  0.5809688568115234
темнота_NOUN :  0.5586055517196655
ночной_ADJ :  0.5421528220176697


человек_NOUN
[-3.8098485   2.7259772   1.1949589  -1.3183012   2.5352347 

Находить косинусную близость пары слов функцией ```similarity()```:

In [8]:
print(model_ru.similarity('человек_NOUN', 'обезьяна_NOUN'))

0.26860568


У загруженной модели много различных функций. Например, можно решать задачи на семантическую близость.

Что получится, если вычесть из пиццы Италию и прибавить Сибирь?

Для решения примера в качестве параметров метода ```most_similar()``` необходимо передать:
* positive — вектора, которые мы складываем
* negative — вектора, которые вычитаем

*Замечание:* не забываем взять самый близкий элемент, для этого необходимо указать ```[0][0]```.

In [9]:
print(model_ru.most_similar(positive=['футбол_NOUN', 'россия_NOUN'], negative=['хоккей_NOUN'])[0][0])

геополитика_PROPN


In [10]:
# придумайте и проверьте с помощью метода most_similar какую-нибудь аналогию
languages = {
key: model_ru.most_similar(positive=['английский_NOUN', key],
negative=['великобритания_NOUN'])[0][0]
for key in ['россия_NOUN', 'франция_NOUN', 'япония_NOUN', 'испания_NOUN']
}

print(languages)

print(model_ru.most_similar(positive=['король_NOUN',
'женщина_NOUN'], negative=['мужчина_NOUN'])[0][0])

{'россия_NOUN': 'русяча_NOUN', 'франция_NOUN': 'франц_NOUN', 'япония_NOUN': 'англ_ADJ', 'испания_NOUN': 'англ_ADJ'}
королева_NOUN


Метод ```doesnt_match()``` находит "лишнее слово" в группе слов:

In [11]:
model_ru.doesnt_match('пицца_NOUN пельмень_NOUN хот-дог_NOUN ананас_NOUN'.split())

'ананас_NOUN'

## Собственная модель

In [12]:
# скачиваем датасет
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
--2025-05-29 19:31:19--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67281491 (64M) [text/plain]
Saving to: 'unlabeledTrainData.tsv.1'

unlabeledTrainData. 100%[===================>]  64.16M  13.8MB/s    in 7.9s    

2025-05-29 19:31:28 (8.17 MB/s) - 'unlabeledTrainData.tsv.1' saved [67281491/67281491]



Нам необходимо отчистить данные от лишнего: убрать ссылки, html-разметку и небуквенные символы. Затем нужно привести все к нижнему регистру и токенизировать.

На выходе мы хотим получить массив из предложений, каждое из которых представляет собой массив слов.

Импортируем необходимые библиотеки и методы:

In [13]:
import pandas as pd
import nltk.data
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, RegexpTokenizer
import gensim

nltk.download('punkt')
nltk.download('punkt_tab')
data = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting =3)
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

[nltk_data] Downloading package punkt to /home/glebmavi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/glebmavi/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Функции для очистки данных:

In [14]:
def review_to_wordlist(review, remove_stopwords=False):
    # убираем ссылки вне тегов
    review = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    # достаем сам текст
    review_text = BeautifulSoup(review, "lxml").get_text()
    # оставляем только буквенные символы
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    if remove_stopwords:
      # убираем стоп-слова
        stops = stopwords.words("english")
        words = [w for w in words if not w in stops]
    return(words)

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

Проходим по всему датасету и парсим написанной выше функцией текст в списки слов, удаляя при этом лишнее:

In [15]:
sentences = []

print("Parsing sentences from training set...")
for review in data["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set...


Посмотрим, что получилось:

In [16]:
print(len(sentences))
print(sentences[0])

529416
['watching', 'time', 'chasers', 'it', 'obvious', 'that', 'it', 'was', 'made', 'by', 'a', 'bunch', 'of', 'friends']


In [17]:
# это понадобится нам позже для обучения другой модели эмбеддингов

with open('clean_text.txt', 'w') as f:
    for s in sentences[:5000]:
        f.write(' '.join(s))
        f.write('\n')

Обучаем и сохраняем модель.


Основные параметры:
* данные должны быть итерируемым объектом
* size — размер вектора,
* window — размер окна наблюдения,
* min_count — мин. частотность слова в корпусе,
* sg — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* sample — порог для downsampling'a высокочастотных слов,
* workers — количество потоков,
* alpha — learning rate,
* iter — количество итераций,
* max_vocab_size — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение превышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

**NB!** Обратите внимание, что тренировка модели не включает препроцессинг! Это значит, что избавляться от пунктуации, приводить слова к нижнему регистру, лемматизировать их, проставлять частеречные теги придется до тренировки модели (если, конечно, это необходимо для вашей задачи). Т.е. в каком виде слова будут в исходном тексте, в таком они будут и в модели.

In [18]:
# обучаем модель с векторами размерности 300, длиной окна 10
model_en = gensim.models.Word2Vec(sentences, workers=4, vector_size=300, min_count=10, window=10, sample=1e-3)

Смотрим, сколько в модели слов.

In [19]:
print(len(model_en.wv))

28308


Попробуем оценить модель вручную, порешав примеры. Несколько дано ниже, попробуйте придумать свои.

In [20]:
print(model_en.wv.most_similar(positive=["woman", "actor"], negative=["man"], topn=1))
print(model_en.wv.most_similar(positive=["dogs", "man"], negative=["dog"], topn=1))

print(model_en.wv.most_similar("usa", topn=3))

print(model_en.wv.doesnt_match("comedy thriller western novel".split()))

[('actress', 0.7681520581245422)]
[('men', 0.6274680495262146)]
[('germany', 0.7383697628974915), ('europe', 0.730600893497467), ('north', 0.7086883783340454)]
novel


In [21]:
print(model_en.wv.most_similar(positive=["hill", "high"], negative=["low"], topn=1))
print(model_en.wv.most_similar(positive=["cats", "woman"], negative=["cat"], topn=1))

print(model_en.wv.most_similar("football", topn=3))

print(model_en.wv.doesnt_match("car train horse bus".split()))

[('avenue', 0.5459960699081421)]
[('women', 0.609978437423706)]
[('basketball', 0.7553985118865967), ('soccer', 0.7295182943344116), ('baseball', 0.6655445098876953)]
horse


### Как дообучить существующую модель

При тренировке модели "с нуля" веса инициализируются случайно, однако, можно использовать для инициализации векторов веса из предобученной модели, таким образом как бы дообучая ее.

Сначала посмотрим близость какой-нибудь пары слов в имеющейся модели, чтобы потом сравнить результат с дообученной.

In [22]:
model_en.wv.similarity('lion', 'rabbit')

0.28527483

В качестве дополнительных данных для обучения возьмем английский текст «Алисы в Зазеркалье».

In [23]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
--2025-05-29 19:33:08--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167631 (164K) [text/plain]
Saving to: 'alice.txt.1'

alice.txt.1         100%[===================>] 163.70K  1.03MB/s    in 0.2s    

2025-05-29 19:33:08 (1.03 MB/s) - 'alice.txt.1' saved [167631/167631]



In [24]:
with open("alice.txt", 'r', encoding='utf-8') as f:
    text = f.read()

# убираем переносы строк, токенизируем текст

text = re.sub('\n', ' ', text)
sents = sent_tokenize(text)

punct = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~„“«»†*—/\-‘’'
clean_sents = []

# убираем всю пунктуацию и делим текст на слова по пробелу
for sent in sents:
    s = [w.lower().strip(punct) for w in sent.split()]
    clean_sents.append(s)

print(clean_sents[:2])

[['through', 'the', 'looking-glass', 'by', 'lewis', 'carroll', 'chapter', 'i', 'looking-glass', 'house', 'one', 'thing', 'was', 'certain', 'that', 'the', 'white', 'kitten', 'had', 'had', 'nothing', 'to', 'do', 'with', 'it', '', 'it', 'was', 'the', 'black', 'kitten’s', 'fault', 'entirely'], ['for', 'the', 'white', 'kitten', 'had', 'been', 'having', 'its', 'face', 'washed', 'by', 'the', 'old', 'cat', 'for', 'the', 'last', 'quarter', 'of', 'an', 'hour', 'and', 'bearing', 'it', 'pretty', 'well', 'considering', 'so', 'you', 'see', 'that', 'it', 'couldn’t', 'have', 'had', 'any', 'hand', 'in', 'the', 'mischief']]


Чтобы дообучить модель, надо сначала ее сохранить, а потом загрузить. Все параметры тренировки (размер вектора, мин. частота слова и т.п.) будут взяты из загруженной модели, т.е. задать их заново нельзя.

In [25]:
model_path = "movie_reviews.model"

# так можно сохранить модель для последующего дообучения
print("Saving model...")
model_en.save(model_path)

Saving model...


In [26]:
from gensim.models import Word2Vec

# загружаем нашу обученную модель и дообучаем на текстах "Алисы"

model = Word2Vec.load(model_path)

model.build_vocab(clean_sents, update=True)
model.train(clean_sents, total_examples=model.corpus_count, epochs=5)

(96950, 150225)

Лев и кролик стали ближе друг к другу!

In [27]:
model.wv.similarity('lion', 'rabbit')

0.29905683

In [28]:
model_path = "movies_alice.bin"

print("Saving model...")
model_en.wv.save_word2vec_format(model_path, binary=True)

Saving model...


## Оценка


In [29]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/evaluation/ru_analogy_tagged.txt

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
--2025-05-29 19:33:13--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/evaluation/ru_analogy_tagged.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 871776 (851K) [text/plain]
Saving to: 'ru_analogy_tagged.txt.1'

ru_analogy_tagged.t 100%[===================>] 851.34K  2.10MB/s    in 0.4s    

2025-05-29 19:33:14 (2.10 MB/s) - 'ru_analogy_tagged.txt.1' saved [871776/871776]



Посчитаем точность модели на датасете с аналогиями:

In [30]:
res = model_ru.evaluate_word_analogies('ru_analogy_tagged.txt')

In [31]:
print(res[0])
print(res[1][0])

for section in res[1]:
    correct = len(section['correct'])
    incorrect = len(section['incorrect'])
    accuracy = correct / (correct + incorrect) if (correct + incorrect) > 0 else 0
    print(f"Section: {section['section']}, Accuracy:{accuracy :.2%}")

0.0
{'section': 'capital-common-countries', 'correct': [], 'incorrect': []}
Section: capital-common-countries, Accuracy:0.00%
Section: capital-world, Accuracy:0.00%
Section: currency, Accuracy:0.00%
Section: city-in-state, Accuracy:0.00%
Section: family, Accuracy:0.00%
Section: gram1-Aective-to-adverb, Accuracy:0.00%
Section: gram2-opposite, Accuracy:0.00%
Section: gram6-nationality-Aective, Accuracy:0.00%
Section: Total accuracy, Accuracy:0.00%


## Визуализация

In [32]:
from nltk import FreqDist
from tqdm.notebook import tqdm
from sklearn.manifold import TSNE

In [33]:
top_words = []

fd = FreqDist()
for s in tqdm(sentences):
    fd.update(s)

for w in fd.most_common(1000):
    top_words.append(w[0])

print(top_words[:50:])
top_words_vec = model.wv[top_words]

  0%|          | 0/529416 [00:00<?, ?it/s]

['the', 'and', 'a', 'of', 'to', 'is', 'it', 'in', 'i', 'this', 'that', 's', 'was', 'as', 'with', 'for', 'movie', 'but', 'film', 'you', 't', 'on', 'not', 'he', 'are', 'his', 'have', 'be', 'one', 'all', 'they', 'at', 'by', 'who', 'an', 'from', 'so', 'like', 'there', 'or', 'her', 'just', 'about', 'out', 'has', 'if', 'what', 'some', 'good', 'can']


In [35]:
top_words_vec = model.wv[top_words]

Применяем преобразование t-SNE для векторов выбранных 1000 слов:

In [36]:
%%time
# инициализируем модель
tsne = TSNE(n_components=2, random_state=0)
# обучаем и применяем
top_words_tsne = tsne.fit_transform(top_words_vec)

CPU times: user 20.1 s, sys: 3.19 s, total: 23.3 s
Wall time: 2.61 s


In [37]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

Чтобы вычислить преобразование t-SNE быстрее (и иногда еще и эффективнее), можно сперва снизить размерность исходных данных с помощью, например, SVD (singular value decomposition, сингулярное разлолжение матрицы), и потом применять t-SNE.

Понизим размерность наших векторов с исходных 300 до 50 с помощью SVD:


In [38]:
# загружаем SVD из библиотеки sklearn
from sklearn.decomposition import TruncatedSVD

# инициализируем SVD
svd_50 = TruncatedSVD(n_components=50)

# обучаем и применяем разложение
top_words_vec_50 = svd_50.fit_transform(top_words_vec)

# обучаем и применяем TSNE
top_words_tsne2 = TSNE(n_components=2, random_state=0).fit_transform(top_words_vec_50)

Нарисуем полученный результат:

In [39]:
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words, +SVD)")

source = ColumnDataSource(data=dict(x1=top_words_tsne2[:,0],
                                    x2=top_words_tsne2[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

## FastText

In [40]:
import fasttext

ft_model = fasttext.train_unsupervised('clean_text.txt', minn=3, maxn=4, dim=300)

Read 0M words
Number of words:  2436
Number of labels: 0
Progress: 100.0% words/sec/thread:   98636 lr:  0.000000 avg.loss:  2.749217 ETA:   0h 0m 0s


Посмотрим на вектор для слова "movie":

In [41]:
ft_model.get_word_vector("movie")

array([-0.03372104, -0.05612415, -0.10509384, -0.08622092,  0.15641013,
       -0.04973717, -0.10229631,  0.1782459 , -0.01056085, -0.06433695,
       -0.00228852, -0.0844503 , -0.05315302,  0.13464028,  0.08346517,
        0.23560286, -0.11725441, -0.14061408,  0.04722716, -0.03544731,
        0.05611718,  0.21291149, -0.05827711,  0.13642135, -0.02270552,
        0.0149653 , -0.07217506, -0.05248531, -0.07062826,  0.00543572,
       -0.17101401,  0.11031697, -0.13429002, -0.05777853, -0.1527026 ,
       -0.02054235, -0.06124336,  0.10353541, -0.05617575,  0.12050416,
        0.2166105 ,  0.075666  , -0.04244894,  0.09138583,  0.02125417,
       -0.14540307,  0.01948881,  0.1850579 ,  0.09291629,  0.13560699,
        0.00543272,  0.06080012, -0.10832422,  0.03530419, -0.03985006,
       -0.0475775 , -0.05726317,  0.01480375,  0.07751747,  0.03421183,
       -0.03747869,  0.06621891,  0.11612202,  0.00843818,  0.1340326 ,
        0.04652696,  0.03155451,  0.12457638, -0.06438967, -0.00

Метод ```get_nearest_neighbors``` возвращает самые похожие слова (аналог метода ```most_similar()``` для w2v):

In [42]:
ft_model.get_nearest_neighbors('actor')

[(0.9998452663421631, 'actors'),
 (0.999701738357544, 'directors'),
 (0.9996725916862488, 'director'),
 (0.9996594786643982, 'actress'),
 (0.9996394515037537, 'act'),
 (0.999450147151947, 'correct'),
 (0.9994181990623474, 'dirty'),
 (0.999415934085846, 'actual'),
 (0.9994115829467773, 'attractive'),
 (0.9993978142738342, 'perspective')]

С помощью метода ```get_analogues()``` можно получить аналогии:

In [43]:
ft_model.get_analogies("woman", "man", "actor")

[(0.9992786645889282, 'act'),
 (0.9992228150367737, 'actors'),
 (0.9992055296897888, 'actress'),
 (0.9990889430046082, 'actually'),
 (0.998973548412323, 'written'),
 (0.9989454746246338, 'bad'),
 (0.9989438056945801, 'recent'),
 (0.9989053606987, 'recently'),
 (0.9988710880279541, 'edited'),
 (0.9988704323768616, 'committed')]

Проблема с опечатками решена!

In [44]:
ft_model.get_nearest_neighbors('actr')

[(0.9996079206466675, 'act'),
 (0.999275803565979, 'actors'),
 (0.9990800619125366, 'actor'),
 (0.9989238977432251, 'actress'),
 (0.9987796545028687, 'directors'),
 (0.9987174868583679, 'acted'),
 (0.9986833333969116, 'actual'),
 (0.9986427426338196, 'directed'),
 (0.9986122250556946, 'direct'),
 (0.9985419511795044, 'acts')]

Проблема с out of vocabulary словами тоже решена!


In [45]:
ft_model.get_nearest_neighbors('moviegeek')

[(0.9996760487556458, 'move'),
 (0.9995264410972595, 'movie'),
 (0.9994995594024658, 'reviews'),
 (0.9994115233421326, 'said'),
 (0.9994101524353027, 'view'),
 (0.9994035363197327, 'thoroughly'),
 (0.9994027018547058, 'good'),
 (0.9993962049484253, 'likely'),
 (0.9993897080421448, 'enjoy'),
 (0.9993826746940613, 'but')]

# Бонус*: Применим полученные выше навыки и решим простую задачу анализа тональности твиттов:

Проделаем весь пайплайн от сырых текстов до получения обученной модели.
Отдельно скачиваем файлы с положительно окрашенными твитами и негативно окрашеннными.
Это реальные данные русскоязычного сегмента твиттера.


In [46]:
!wget -O positive.csv https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
--2025-05-29 19:39:48--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/6mg7rw3wltux83q2o4ah4/positive.csv?rlkey=cvruhzofza9kkfxwzyp2vskfd&dl=0 [following]
--2025-05-29 19:39:49--  https://www.dropbox.com/scl/fi/6mg7rw3wltux83q2o4ah4/positive.csv?rlkey=cvruhzofza9kkfxwzyp2vskfd&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc83d2df4510cd794c1f5da68491.dl.dropboxusercontent.com/cd/0/inline/Cqn033mb7wBH27zCgMtvdStf0rw86h4NcJ1sD590ZhHotYUIMzXZEjl0q345osjLOwqPFLghUdUPF1-I2gDn9Aiige7Vao0dkDb159Zx6Icvguw6HTfs7fl-Mnf2FHoz5Bk/file# [following]
--2025-05-29 19:39:49--  https

In [47]:
!wget -O negative.csv https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
--2025-05-29 19:39:57--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.69.18, 2620:100:6025:18::a27d:4512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.69.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/wui0xz78kpna56690uej4/negative.csv?rlkey=309xeou9u3rtbejw9stb13wfr&dl=0 [following]
--2025-05-29 19:39:57--  https://www.dropbox.com/scl/fi/wui0xz78kpna56690uej4/negative.csv?rlkey=309xeou9u3rtbejw9stb13wfr&dl=0
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc26ffb2d4d926b5c26ba7dd1fb1.dl.dropboxusercontent.com/cd/0/inline/Cql310tSgxCJZ0sEOmJSF2qJ2i_a8RxNS7EsjBVB_YKrMxhBr2bF5eOkqHr8ZKGmhqa92EKBdenJDXkVlMvAwdggTaqs1BX-Y1kOEb21oWe_KyglDAVRiiF7vTIblN1a7NU/file# [following]
--2025-05-29 19:39:58--  https

In [49]:
import pandas as pd

positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)

negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = pd.concat([positive , negative])
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


In [50]:
len(df)

226834

Проведем стандартный препроцессинг:

In [51]:
import pymorphy2
from multiprocessing import Pool
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
import re

m = pymorphy2.MorphAnalyzer()

# убираем все небуквенные символы
regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text)
    except:
        return []

In [52]:
def lemmatize(text, pymorphy=m):
    try:
        return " ".join([pymorphy.parse(w)[0].normal_form for w in text])
    except:
        return " "

In [53]:
def clean_text(text):
    return lemmatize(words_only(text))

In [54]:
with Pool(8) as p:
    lemmas = list(tqdm(p.imap(clean_text, df['text']), total=len(df)))

df['lemmas'] = lemmas
df.head()

/tmp/ipykernel_55556/305022119.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  lemmas = list(tqdm(p.imap(clean_text, df['text']), total=len(df)))


  0%|          | 0/226834 [00:00<?, ?it/s]

,text,label,lemmas
0,"@first_timee хоть я и школота, но поверь, у на...",positive,first_timee хоть я и школотый но поверь у мы т...
1,"Да, все-таки он немного похож на него. Но мой ...",positive,да всё таки он немного похожий на он но мой ма...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive,rt katiacheh: ну ты идиотка) я испугаться за т...
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive,rt digger : кто то в угол сидеть и погибать от...
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive,irina_dyshkant вот что значит страшилка :d но ...


Запишем полученные данные в формате для обучения классификатора:

In [55]:
# переводим данные из датафрейма в списки

X = df.lemmas.tolist()
y = df.label.tolist()

X, y = np.array(X), np.array(y)

# разбиваем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)
print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

total train examples 151978
total test examples 74856


Записываем train и test выборки в файлы в соответствии с форматом выше:

In [56]:
with open('data.train.txt', 'w+') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')


with open('test.txt', 'w+') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

Обучаем классификатор fasttext:

In [57]:
classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')

Read 2M words
Number of words:  239875
Number of labels: 2
Progress: 100.0% words/sec/thread:  950348 lr:  0.000000 avg.loss:  0.182824 ETA:   0h 0m 0s


Смотрим на метрики качества (precision и recall) полученной модели:

In [58]:
print('P@1:', result[1])
print('R@1:', result[2])
print('Number of examples:', result[0])

P@1: 0.8972427059955114
R@1: 0.8972427059955114
Number of examples: 74856
